In [1]:
import cv2
from ultralytics import YOLO
import json
import os
import datetime
import numpy as np
from collections import OrderedDict
from queue import Queue
import threading

In [2]:
# Load the YOLO11 model
model = YOLO("yolo11n.pt")

In [3]:
# Open the video file
def load_config(config_path):
    with open(config_path, 'r') as f:
        return json.load(f)

configstr = "config2.json"
config = load_config(configstr)
 
video_path = config.get("video_path")
print(video_path)

F:/python/karamouzi/saving_frames2/video/vid2.mp4


In [4]:
#save frames
saved_objects = OrderedDict()
save_path = config.get("save_path")
now = datetime.datetime.now()
date_str = now.strftime("%Y/%m/%d")
dir_path = os.path.join(save_path, date_str)
os.makedirs(dir_path, exist_ok=True)

def calculate_sharpness(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)    
    return laplacian.var()

save_queue = Queue(maxsize=500)

def manage_saved_objects():
    max_saved_objects = config.get("max_saved_objects")
    if len(saved_objects) > max_saved_objects:
        oldest_key = next(iter(saved_objects))
        del saved_objects[oldest_key]

def save_worker():
    while True:
        item = save_queue.get()
        if item is None:
            break
        cv2.imwrite(file_path, frame)
        save_queue.task_done()

save_thread = threading.Thread(target=save_worker, daemon=True)
save_thread.start()

def enqueue_save(frame, file_path):
    try:
        save_queue.put_nowait((frame, file_path)) 
    except queue.Full:
        print("error")
        
def save_frames(r, frame, frameid):
    for box in r.boxes:
        class_id = int(box.cls.item())
        track_id = int(box.id) if box.id is not None else None
        min_confidence = config.get("min_confidence")
        confidence = box.conf.item()
        if track_id is None:
            continue
        if confidence < min_confidence:
            continue
        
        name_saved = f"P_{track_id}.jpg" if class_id == 0 else f"V_{track_id}.jpg"
        file_path = os.path.join(dir_path , name_saved)
        
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
#        cropped_object = frame[y1:y2, x1:x2]
        current_sharpness = calculate_sharpness(frame)
        min_sharpness = config.get("min_sharpness")
        
        if track_id not in saved_objects or current_sharpness > saved_objects[track_id]['sharpness']:
            saved_objects[track_id] = {"sharpness": current_sharpness, "file_path": file_path}
            #cv2.imwrite(file_path, frame)
            
            color = (0, 255, 0) if class_id == 0 else (255, 0, 0) 
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            label = f"ID: {track_id}, Conf: {confidence:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            enqueue_save(frame, file_path)
            
        manage_saved_objects()

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\Sara\AppData\Local\Programs\Python\Python39\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Sara\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Sara\AppData\Local\Programs\Python\Python39\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Sara\AppData\Local\Temp\ipykernel_9632\124243590.py", line 27, in save_worker
NameError: name 'file_path' is not defined


In [5]:
cap = cv2.VideoCapture(video_path)
# Loop through the video frames
frameid = 0
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, classes=[0, 1, 2, 3, 5, 7], show=False)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()
        
        for r in results:
            save_frames(r, frame, frameid)
            frameid += 1
            if cv2.waitKey(10) == 27:                     # exit if Escape is hit
                break                                        

        # Display the annotated frame
        cv2.imshow("YOLO11 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()
save_queue.put(None)
save_thread.join()


0: 384x640 1 car, 156.2ms
Speed: 7.8ms preprocess, 156.2ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 79.0ms
Speed: 5.7ms preprocess, 79.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 78.8ms
Speed: 4.8ms preprocess, 78.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 87.8ms
Speed: 6.6ms preprocess, 87.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 91.6ms
Speed: 4.4ms preprocess, 91.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 91.4ms
Speed: 4.1ms preprocess, 91.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 88.6ms
Speed: 6.8ms preprocess, 88.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 88.3ms
Speed: 5.6ms preprocess, 88.3ms inference, 1.2ms 